# How to use the MLencoding class

This is a tutorial of how to use our MLencoding package to build encoding models a predict spikes. 


In [3]:
import warnings

import numpy as np
import pandas as pd
import scipy.io

##### Load encoding package

In [1]:
from MLencoding import *

# 1. Data
Below we load a dataset available on CRCNS: a [Macaque M1](http://crcns.org/data-sets/movements/dream/downloading-dream) (from [Stevenston et al. 2011](http://jn.physiology.org/content/106/2/764.short)).

The data has been organized in Matlab into neat arrays for easy loading here.

We will soon want a single numpy array representing the external covariates, and a single numpy vector representing the neural response. The data array X will be of dimensions (n, p), where n is the number of time bins and p is the number of covariates, and the response y will be of dimensions (n, ) . We use pandas as an intermediate tool for data organizing, but it's really not necessary - if using your own data just wrangle it into numpy arrays of proper dimension.

#### Load data

In [4]:
m1_imported = scipy.io.loadmat('../data/m1_stevenson_2011.mat')

### 1.1 Covariates

Pull into pandas dataframe. This allows us to easily access covariates by name.

In [5]:
data = pd.DataFrame()
data['time'] =  m1_imported['time'][0]
data['handPos_x'] =  m1_imported['handPos'][0]
data['handPos_y'] =  m1_imported['handPos'][1]
data['handVel_x'] =  m1_imported['handVel'][0]
data['handVel_y'] =  m1_imported['handVel'][1]

#### Compute more covariates/features

#These will be used as the 'engineered' features for improving the GLM's performance.

data['velDir'] = np.arctan2(data['handVel_y'], data['handVel_x'])
data['cos_velDir'] = np.cos(data['velDir'])
data['sin_velDir'] = np.sin(data['velDir'])
data['speed'] = np.sqrt(data['handVel_x'].values**2+data['handVel_y'].values**2)
r = np.arctan2(data['handPos_y'], data['handPos_x'])
data['cos_PosDir'] = np.cos(r)
data['sin_PosDir'] = np.sin(r)
data['radial_Pos'] = np.sqrt(data['handPos_x'].values**2+data['handPos_y'].values**2)
data.head()

,time,handPos_x,handPos_y,handVel_x,handVel_y,velDir,cos_velDir,sin_velDir,speed,cos_PosDir,sin_PosDir,radial_Pos
0,12.591,0.002905,-0.303636,-0.011201,-0.006237,-2.633523,-0.873685,-0.486491,0.012820,0.009568,-0.999954,0.303650
1,12.641,0.002260,-0.303869,-0.010743,-0.000833,-3.064245,-0.997010,-0.077271,0.010775,0.007437,-0.999972,0.303877
2,12.691,0.002399,-0.303631,0.017680,0.012094,0.599956,0.825360,0.564606,0.021420,0.007900,-0.999969,0.303641
3,12.741,0.004010,-0.302399,0.044667,0.038700,0.713933,0.755792,0.654812,0.059100,0.013258,-0.999912,0.302426
4,12.791,0.006386,-0.300673,0.042202,0.017021,0.383375,0.927408,0.374053,0.045505,0.021233,-0.999775,0.300741


# 2. Making an encoding model

We instantiate the object like this:

In [4]:
glm_model = MLencoding(tunemodel = 'glm')

NameError: name 'MLencoding' is not defined

We can then train it on some data. Let's go for 3/4 of the data we have for some neuron.

In [8]:
neuron_n = 1

X = data[['handPos_x','handPos_y','handVel_x','handVel_y']].values
y = m1_imported['spikes'][neuron_n]

n_samples = X.shape[0]
threefourths = int(n_samples*3/4)

X_train = X[:threefourths,:]
y_train = y[:threefourths]


In [8]:
# Now we train the model

glm_model.fit(X_train,y_train)

/Users/mahdiyarshahbazi/Documents/GitHub/spykesML/MLencoding/mlencoding.py:228: UserWarning: 
  Using default hyperparameters. Consider optimizing on a held-out dataset using, e.g. hyperopt or random search
  warnings.warn('\n  Using default hyperparameters. Consider optimizing on'+


AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

Let's predict the neural response on the training set.

In [8]:
X_test = X[threefourths:,:]
y_test = y[threefourths:]

y_hat = glm_model.predict(X_test)

NameError: name 'glm_model' is not defined

How did we do? We can score this prediction with the class's internal function 'poisson_pseudoR2'.

In [11]:
# The 'null model' we measure against is the mean of the train dataset. 
y_null = np.mean(y_train)

pr2_glm = glm_model.poisson_pseudoR2(y_test, y_hat, y_null)
print(pr2_glm)

0.0625913964434


## Cross-validation

Let's now obtain the predictions and scores of 10-fold cross-validation for a GLM.

In [18]:
Y_hat, PR2s = glm_model.fit_cv(X,y, n_cv = 10, verbose = 2)

...runnning cv-fold 1 of 10
pR2:  0.0488023178838
...runnning cv-fold 2 of 10
pR2:  0.0434830590622
...runnning cv-fold 3 of 10
pR2:  0.0513488923378
...runnning cv-fold 4 of 10
pR2:  0.0521074580784
...runnning cv-fold 5 of 10
pR2:  0.0449312912574
...runnning cv-fold 6 of 10
pR2:  0.062685886475
...runnning cv-fold 7 of 10
pR2:  0.0459586387009
...runnning cv-fold 8 of 10
pR2:  0.0578141187789
...runnning cv-fold 9 of 10
pR2:  0.0523027349251
...runnning cv-fold 10 of 10
pR2:  0.0496125678667
pR2_cv: 0.050905 (+/- 0.001765)


### Other methods: neural networks, random forest, XGBoost

Using other encoding models is as simple as this:

In [6]:
nn_model = MLencoding(tunemodel='feedforward_nn')

In [9]:
Y_hat, PR2s = nn_model.fit_cv(X,y, n_cv = 10, verbose = 2)

...runnning cv-fold 1 of 10


/Users/mahdiyar/Documents/GitHub/spykesML/MLencoding/mlencoding.py:228: UserWarning: 
  Using default hyperparameters. Consider optimizing on a held-out dataset using, e.g. hyperopt or random search
  warnings.warn('\n  Using default hyperparameters. Consider optimizing on'+
/opt/miniconda3/envs/test/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step
pR2:  0.1321463832890698
...runnning cv-fold 2 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step
pR2:  0.1218188378451821
...runnning cv-fold 3 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step
pR2:  0.13964401049801722
...runnning cv-fold 4 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step
pR2:  0.1508302165267038
...runnning cv-fold 5 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step
pR2:  0.1239192978540854
...runnning cv-fold 6 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step
pR2:  0.14554531989956
...runnning cv-fold 7 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step
pR2:  0.1478227389380059
...runnning cv-fold 8 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step
pR2:  0.15350352729155514
...runnning cv-fold 9 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step
pR2:  0.14113829075640272
...runnning cv-fold 10 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step
pR2:  0.11119833905131382
pR2_cv: 0.136757 (+/- 0.004205)


### Predicting spikes using spike or covariate history

MLencoding supports models that also use previous covariate values to predict the current spike rate. Spike history is also supported.

When you instantiate a model with the `spike_history=True` or `cov_history=True` keywords, all future calls to `fit`, `predict`, and `fit_cv` will automatically construct a new covariate matrix with additional columns. These columns represent the covariate history. This matrix is then used for fitting.

Currently, covariate history columns are raised cosine basis functions. You can define how many temporal basis you want with `n_filters`, which will span the interval [0, `max_time`]. Times are measured in milliseconds. In order to perform this calculation, the model needs to know how many milliseconds are in each time bin. (Set this with `window`).

In [11]:
xgb_history = MLencoding(tunemodel = 'xgboost',
                         cov_history = False, spike_history=True, # We can choose!
                         window = 50, #this dataset has 50ms time bins
                         n_filters = 2,
                         max_time = 250 )

In [12]:
xgb_history.fit_cv(X,y, verbose = 2, continuous_folds = True);

...runnning cv-fold 0 of 10


/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [14:55:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators", "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


pR2:  0.17312526557608343
...runnning cv-fold 1 of 10
pR2:  0.15354027869384146
...runnning cv-fold 2 of 10
pR2:  0.1867991214326713
...runnning cv-fold 3 of 10


/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [14:55:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators", "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


pR2:  0.1481805093120826
...runnning cv-fold 4 of 10
pR2:  0.12918437656810422
...runnning cv-fold 5 of 10


/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [14:55:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators", "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


pR2:  0.14613716122903364
...runnning cv-fold 6 of 10
pR2:  0.2316779167801466
...runnning cv-fold 7 of 10
pR2:  0.2649427161421759
...runnning cv-fold 8 of 10


/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [14:55:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators", "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


pR2:  0.2766313536015562
...runnning cv-fold 9 of 10
pR2:  0.2673691235257448
pR2_cv: 0.197759 (+/- 0.017099)


Here is a version that uses spike history with random folds.

In [13]:
# First we need to set n_every > max_time/window. 
xgb_history_rand = MLencoding(tunemodel = 'xgboost',
                         cov_history = False, spike_history=True,
                         window = 50, 
                         n_filters = 2,
                         max_time = 250, n_every = 6 )

xgb_history_rand.fit_cv(X,y, verbose = 2, continuous_folds = False);

...runnning cv-fold 1 of 10


/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [14:55:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators", "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


pR2:  0.1681783369025721
...runnning cv-fold 2 of 10


/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [14:55:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators", "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


pR2:  0.20468717331846098
...runnning cv-fold 3 of 10
pR2:  0.18745284098565151
...runnning cv-fold 4 of 10
pR2:  0.13811429056674907
...runnning cv-fold 5 of 10


/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [14:55:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators", "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


pR2:  0.18653838817856427
...runnning cv-fold 6 of 10
pR2:  0.15906367040655156
...runnning cv-fold 7 of 10
pR2:  0.12510741176648033
...runnning cv-fold 8 of 10


/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [14:55:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "n_estimators", "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


pR2:  0.18535515554299553
...runnning cv-fold 9 of 10
pR2:  0.15141831529782124
...runnning cv-fold 10 of 10
pR2:  0.07598434134416732
pR2_cv: 0.158190 (+/- 0.011385)


## Fitting an LSTM

There's nothing special about fitting an LSTM in our implementation. Just be sure to set `spike_history=True` and `cov_history = True`, and to use continuous CV folds.

In [14]:
lstm = MLencoding(tunemodel = 'lstm',
                         cov_history = True, spike_history=True, # We can choose!
                         window = 50, #this dataset has 50ms time bins
                         n_filters = 4,
                         max_time = 250 )

In [15]:
lstm.fit_cv(X,y, verbose = 2, continuous_folds = True);

...runnning cv-fold 0 of 10


/Users/mahdiyarshahbazi/Documents/GitHub/spykesML/MLencoding/mlencoding.py:228: UserWarning: 
  Using default hyperparameters. Consider optimizing on a held-out dataset using, e.g. hyperopt or random search
  warnings.warn('\n  Using default hyperparameters. Consider optimizing on'+
/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.16531543658499015
...runnning cv-fold 1 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.13604802743565358
...runnning cv-fold 2 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.18528197812044578
...runnning cv-fold 3 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.1548746585687506
...runnning cv-fold 4 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.12418895237961858
...runnning cv-fold 5 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.14886382095399808
...runnning cv-fold 6 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.25077728576605285
...runnning cv-fold 7 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.26962244882129416
...runnning cv-fold 8 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.2448194816505176
...runnning cv-fold 9 of 10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
pR2:  0.2870709083171117
pR2_cv: 0.196686 (+/- 0.018115)


### Getting and setting model parameters
To get the current set of parameters, we can either run:

In [16]:
nn_model.params
# or nn_model.get_params()

{'dropout': 0.05,
 'l2': 1.6e-08,
 'lr': 0.001,
 'n1': 76,
 'n2': 16,
 'decay': 0.009,
 'clipnorm': 1.3,
 'b1': 0.2,
 'b2': 0.02}

We can set the parameters with the `set_params` method. This method takes a dictionary, which update the current set of parameters used.

In [17]:
nn_model.set_params({'dropout':0.3})
nn_model.params

{'dropout': 0.3,
 'l2': 1.6e-08,
 'lr': 0.001,
 'n1': 76,
 'n2': 16,
 'decay': 0.009,
 'clipnorm': 1.3,
 'b1': 0.2,
 'b2': 0.02}

### Hyperparameter optimization using hyperopt

We might not want the default parameters. Here's how to set some better ones

In [ ]:
from hyperopt import fmin, hp, Trials, tpe, STATUS_OK

# Makes sure these are in nn_models.params, otherwise you'll get a key error
space4rf = {
    'dropout': hp.uniform('dropout', 0., 0.6),
    'n1': hp.uniform('n1', 2,128),
    'n2': hp.uniform('n2', 1,15),
}

#object that holds iteration results
trials = Trials()

#define model
nn_model = MLencoding(tunemodel='feedforward_nn')

#function to minimize
def fnc(params):
    
    # make sure parameters are integers that need to be. 
    params['n1'] = int(params['n1'])
    params['n2'] = int(params['n2'])

    nn_model.set_params(params)
    
    # Remember that X and y have been defined above.
    Y_hat, PR2s = nn_model.fit_cv(X,y, n_cv = 5, verbose = 0)

    # return negative since hyperopt always minimizes the function
    return -np.mean(PR2s)

Let's assume that our neuron #1 is a held-out neuron for parameter optimization. Let's optimize:

In [21]:
hyperoptBest = fmin(fnc, space4rf, algo=tpe.suggest, max_evals=50, trials=trials)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/mahdiyarshahbazi/miniconda3/envs/test/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



 1/98 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step

 1/98 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step

 1/98 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step

  0%|          | 0/50 [00:11<?, ?trial/s, best loss=?]

job exception: name 'pseudo_R2' is not defined



  0%|          | 0/50 [00:11<?, ?trial/s, best loss=?]


NameError: name 'pseudo_R2' is not defined

# Defining your own models

The `MLencoding` class is flexible and can be used with predefined models as long as they have `fit` and `predict` methods.

Let's build a different type of neural network, for example.

In [30]:
my_model = Sequential()
my_model.add(Dense(100, input_dim=np.shape(X)[1], init='glorot_normal',
            activation='relu',))
my_model.add(Dense(1,activation='softplus'))
optim = Nadam()
my_model.compile(loss='poisson', optimizer=optim,)


In [31]:
my_enc = MLencoding(tunemodel = my_model)

In [33]:
my_enc.fit_cv(X,y,n_cv=5,verbose=2);

...runnning cv-fold 1 of 5
pR2:  -0.00401729001754
...runnning cv-fold 2 of 5
pR2:  -0.00440856722819
...runnning cv-fold 3 of 5
pR2:  -0.00344133554292
...runnning cv-fold 4 of 5
pR2:  -0.000698628352245
...runnning cv-fold 5 of 5
pR2:  -0.00209311949187
pR2_cv: -0.002932 (+/- 0.000610)


This model isn't great, but you see how it's possible. 

There are some limitations here, though. One thing I can think off the bat is that this Keras model won't work if we set `spike_history = True`, since that will change the shape of `X` and the shape of the input layer is hard-coded when we built this model. 